<a href="https://colab.research.google.com/github/cmiyachi/AI-Yogini-Project/blob/master/project_1/lm_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1: Build an LLM Playground

Welcome to your first project! In this project, you'll build a simple large language model (LLM) playground, an interactive environment where you can experiment with LLMs and understand how they work under the hood.

The goal here is to understand the foundations and mechanics behind LLMs rather than relying on higher-level abstractions or frameworks. You'll see what happens “under the hood”, how an LLM receives a text, processes it, and generate a response. In later projects, you'll use frameworks like Ollama and LangChain that simplify many of these steps. But before that, this project will help you build a solid mental model of how LLMs actually work.

We'll use Google Colab, a free browser-based platform that lets you run Python code and machine learning models without installing anything locally. Click the button below to open this notebook in Colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bytebyteai/ai-eng-projects-2/blob/main/project_1/lm_playground.ipynb)

If you prefer to run the project locally, you can use the provided `env.yaml` file to create a compatible environment using conda. To do so, open a terminal in the same directory as this notebook and run:

```bash
# Create and activate the conda environment
conda env create -f env.yaml && conda activate llm_playground

# Register this environment as a Jupyter kernel
python -m ipykernel install --user --name=llm_playground --display-name "llm_playground"
```


---
## Learning Objectives  
- Understand tokenization and how raw text is converted into a sequence of discrete tokens
- Inspect GPT-2 and the Transformer architecture
- Learn how to load pretrained LLMs using Hugging Face
- Explore decoding strategies to generate text from LLMs
- Compare completion models with instruction-tuned models


Let's get started!

In [ ]:
# Confirm required libraries are installed and working.
import torch, transformers, tiktoken
print("torch", torch.__version__, "| transformers", transformers.__version__)
print("✅ Environment check complete. You're good to go!")

# 1 - Tokenization

A neural network cannot process raw text directly. It needs numbers.
Tokenization is the process of converting text into numerical IDs that models can understand. In this section, you will learn how tokenization works in practice and why it is an essential step in every language model pipeline.

Tokenization methods generally fall into three main categories:
1. Word-level
2. Character-level
3. Subword-level

### 1.1 - Word-level tokenization
This method splits text by whitespace and treats each word as a single token. In the next cell, you will implement a basic word-level tokenizer by building a vocabulary that maps words to IDs and writing `encode` and `decode` functions.

In [ ]:
# Creating a tiny corpus. In practice, a corpus is generally the entire internet-scale dataset used for training.
corpus = [
    "The quick brown fox jumps over the lazy dog",
    "Tokenization converts text to numbers",
    "Large language models predict the next token"
]

# Step 1: Build vocabulary (all unique words in the corpus) and mappings
vocab = []
word2id = {}
id2word = {}

# Step 1: Build vocabulary (all unique words in the corpus) and mappings
for sentence in corpus:
    for word in sentence.split():
        if word not in word2id:
            word2id[word] = len(vocab)  # assign next index
            vocab.append(word)

# Build reverse mapping
id2word = {idx: word for word, idx in word2id.items()}

"""
YOUR CODE HERE (~6-15 lines of code)
"""

print(f"Vocabulary size: {len(vocab)} words")
print("First 15 vocab entries:", vocab[:15])


In [ ]:
# Step 2: Define encode and decode functions
def encode(text):
    # converts text to token IDs
    return [word2id[word] for word in text.split() if word in word2id]
    """
    YOUR CODE HERE (~1-5 lines of code)
    """



def decode(ids):
    # converts token IDs back to text
    return " ".join([id2word[i] for i in ids])
    """
    YOUR CODE HERE (~1-5 lines of code)
    """

In [ ]:
# Step 3: Test your tokenizer with random sentences.
# Try a sentence with unseen words and see what happens (and how to fix it)

sample_text = "The quick brown fox"
encoded = encode(sample_text)
print("Encoded:", encoded)

decoded = decode(encoded)
print("Decoded:", decoded)

sample_text = "hello my name is fox"
encoded = encode(sample_text)
print("uknown ", encoded)

While word-level tokenization is simple and easy to understand, it has two key limitations that make it impractical for large-scale models:
1.  large vocabulary size: every new word or variation (for example, run, runs, running) increases the total vocabulary, leading to higher memory and training costs.
2. Out-of-vocabulary (OOV) problem: the model cannot handle unseen or rare words that were not part of the training vocabulary, so they must be replaced with a generic [UNK] token.

The next section introduces character-level tokenization, where text is represented as individual characters instead of words.

### 1.2 - Character-level tokenization

In this approach, every single character (including spaces, punctuation, and even emojis) is assigned its own ID.

In the next section, we will rebuild a tokenizer using the same corpus as before, but this time with a character-level approach.
For simplicity, assume we are only using lowercase and uppercase English letters (a-z, A-Z).

In [ ]:
import string

# Step 1: Create a vocabulary that includes all uppercase and lowercase letters.
vocab = []
char2id = {}
id2char = {}

# Step 1: Create a vocabulary that includes all uppercase and lowercase letters.
vocab = list(string.ascii_lowercase + string.ascii_uppercase)  # 26 lowercase + 26 uppercase
vocab = ['<PAD>', '<UNK>'] + vocab  # Add two special tokens at the start

# Create mappings
char2id = {char: idx for idx, char in enumerate(vocab)}
id2char = {idx: char for idx, char in enumerate(vocab)}


print(f"Vocabulary size: {len(vocab)} (52 letters + 2 specials)")


In [ ]:
# Step 2: Implement encode() and decode() functions to convert between text and IDs.
def cencode(text):
    # convert text to list of IDs
    return [char2id.get(char, char2id['<UNK>']) for char in text]


def cdecode(ids):
    # Convert list of IDs to text
    return ''.join([id2char.get(i, '<UNK>') for i in ids])

In [ ]:
# Step 3: Test your tokenizer on a short sample word.
# Step 3: Test your tokenizer on a short sample word
sample_text = "Hello!@"
encoded = cencode(sample_text)
decoded = cdecode(encoded)

print(f"Original text: {sample_text}")
print(f"Encoded IDs: {encoded}")
print(f"Decoded text: {decoded}")

Character-level tokenization solves the out-of-vocabulary problem but introduces new challenges:

1. Longer sequences: because each word becomes many tokens, models need to process much longer inputs.
2. Weaker semantic representation: individual characters carry very little meaning, so models must learn relationships across many steps.
3. Higher computational cost: longer sequences lead to more tokens per input, which increases training and inference time.

To find a better balance between vocabulary size and sequence length, we move to subword-level tokenization next.

### 1.3 - Subword-level tokenization

Sub-word methods such as `Byte-Pair Encoding (BPE)`, `WordPiece`, and `SentencePiece` **learn** common groups of characters and merge them into tokens. For example, the word **unbelievable** might turn into three tokens: **["un", "believ", "able"]**. This approach strikes a balance between word-level and character-level methods and fix their limitations.

The BPE algorithm builds a vocabulary iteratively using the following process:
1. Start with individual characters (each character is a token).
2. Count all adjacent pairs of tokens in a large text corpus.
3. Merge the most frequent pair into a new token.

Repeat steps 2 and 3 until you reach the desired vocabulary size (for example, 50,000 tokens).

In the next cell, you will experiment with BPE in practice to see how it compresses text into meaningful subword units. Instead of implementing the algorithm from scratch, you will use a pretrained tokenizer, which was already trained on a large text corpus to build its vocabulary, such as the data used to train `GPT-2`. This allows you to see how BPE works in practice with a real, learned vocabulary.

In [ ]:
from transformers import AutoTokenizer

# Step 1: Load a pretrained GPT-2 tokenizer from Hugging Face.
# Refer to this to learn more: https://huggingface.co/docs/transformers/en/model_doc/gpt2
tokenizer = AutoTokenizer.from_pretrained("gpt2")


In [ ]:
# Step 2: Use it to write encode and decode helper functions
def encode(text):
    return tokenizer.encode(text, add_special_tokens=True)


def decode(ids):
    return tokenizer.decode(ids)

In [ ]:
# 3. Inspect the tokens to see how BPE breaks words apart.
sample = "Unbelievable tokenization powers! 🚀"
tokens = tokenizer.tokenize(sample)
print(tokens)

### 1.4 - TikToken

`tiktoken` is a fast, production-ready library for tokenization used by OpenAI models.
It is designed for efficiency and consistency with how OpenAI counts tokens in GPT models.

In this section, you will explore how different model families use different tokenizers. We will compare tokenizers used to train `GPT-2` and more powerful models such as `GPT-4`. By trying both, you will see how tokenization has evolved to handle more diverse text (including emojis, Unicode, and special characters) while remaining efficient.

In the next cell, you will use tiktoken to load these encodings and inspect how each one splits the same text. You may find reading this doc helpful: https://github.com/openai/tiktoken

In [ ]:
import tiktoken

# Step 1: Load two tokenizers
enc_gpt2 = tiktoken.get_encoding("gpt2")
enc_gpt4 = tiktoken.encoding_for_model("gpt-4")

# Step 2: Encode the same sentence with both and observe how they differ
sentence = "The 🌟 star-programmer implemented AGI overnight."

# Encode with GPT-2
tokens_gpt2 = enc_gpt2.encode(sentence)
print("GPT-2 token IDs:", tokens_gpt2)
print("GPT-2 token count:", len(tokens_gpt2))

# Encode with GPT-4
tokens_gpt4 = enc_gpt4.encode(sentence)
print("\nGPT-4 token IDs:", tokens_gpt4)
print("GPT-4 token count:", len(tokens_gpt4))

# Optional: Decode back to verify
print("\nDecoded GPT-2:", enc_gpt2.decode(tokens_gpt2))
print("Decoded GPT-4:", enc_gpt4.decode(tokens_gpt4))


# Try different sentences
sentences = [
    "Hello world! 😊",  # Emojis and punctuation
    "def hello_world(): print('Hello!')",  # Code snippet
    "Bonjour, comment ça va ?",  # French
    "こんにちは世界",  # Japanese
    "مرحبا بالعالم",  # Arabic
]

for sentence in sentences:
    print(f"\nSentence: {sentence}")

    # GPT-2 encoding
    tokens_gpt2_ids = enc_gpt2.encode(sentence)
    tokens_gpt2_text = [enc_gpt2.decode([tid]) for tid in tokens_gpt2_ids]
    print("\nGPT-2:")
    print("Token IDs:", tokens_gpt2_ids)
    print("Tokens:", tokens_gpt2_text)
    print("Token count:", len(tokens_gpt2_ids))

    # GPT-4 encoding
    tokens_gpt4_ids = enc_gpt4.encode(sentence)
    tokens_gpt4_text = [enc_gpt4.decode([tid]) for tid in tokens_gpt4_ids]
    print("\nGPT-4:")
    print("Token IDs:", tokens_gpt4_ids)
    print("Tokens:", tokens_gpt4_text)
    print("Token count:", len(tokens_gpt4_ids))

Try changing the input sentence and observe how different tokenizers behave.
Experiment with:
- Emojis, special characters, or punctuation
- Code snippets or structured text
- Non-English text (for example, Japanese, French, or Arabic)

If you are curious, you can also attempt to implement the BPE algorithm yourself using a small text corpus to see how token merges are learned in practice.

### 1.5 - Key Takeaways
- **Word-level**: simple and intuitive, but limited by large vocabularies and out-of-vocabulary issues
- **Character-level**: flexible and covers all text, but produces long sequences that are harder to model
- **Subword / BPE**: balances both worlds and is the default choice for most modern LLMs
- **TikToken**: a production-ready tokenizer used in OpenAI models, demonstrating how optimized subword vocabularies are applied in real systems

# 2. What is a Language Model?

At its core, a **language model (LM)** is just a *very large* mathematical function built from many neural-network layers.  
Given a sequence of tokens `[t₁, t₂, …, tₙ]`, it learns to output a probability for the next token `tₙ₊₁`.


Each layer performs basic mathematical operations such as matrix multiplication and attention. When hundreds of these layers are stacked together, the model learns complex patterns and statistical relationships in text. The final output is a vector of scores that represents how likely each possible token is to appear next. You can think of the entire model as one giant equation whose parameters were optimized during training to minimize prediction errors.

### 2.1 - A Single `Linear` Layer

Before jumping into Transformers, let's start with the simplest building block: a `Linear` layer.

A Linear layer computes `y = Wx + b`.

Where:  
  * `x` - input vector  
  * `W` - weight matrix (learned)  
  * `b` - bias vector (learned)

Although this operation looks simple, stacking many linear layers (along with nonlinear activation functions) allows neural networks to model highly complex relationships in data.

In the next cell, you will explore how a **Linear layer** works in practice by implementing one from scratch. You will define the weights and bias, then perform the matrix multiplication and addition manually to see what happens inside this layer. You may find the following links useful:
- https://docs.pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html
- https://docs.pytorch.org/docs/stable/generated/torch.randn.html
- https://docs.pytorch.org/docs/stable/generated/torch.matmul.html

In [ ]:
import torch
import torch.nn as nn

# Define a MyLinear PyTorch module and perform y = Wx + b.

class MyLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(MyLinear, self).__init__()
        # Initialize weights and bias as learnable parameters.
        self.weight = nn.Parameter(torch.randn(out_features, in_features))  # W
        self.bias = nn.Parameter(torch.randn(out_features))  # b

    def forward(self, x):
        # Matrix multiplication followed by bias addition
        return torch.matmul(self.weight, x) + self.bias


lin = MyLinear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))

Next, you will use PyTorch's built-in nn.Linear module, which performs the same computation `(y = Wx + b)` but automatically handles parameter initialization, gradient tracking, and integration with the rest of a neural network. Comparing your manual implementation with this built-in version will help you understand what a linear layer does and how deep learning frameworks make these operations easier to use.

You may find this link useful:
- https://docs.pytorch.org/docs/stable/generated/torch.nn.Linear.html

In [ ]:
import torch.nn as nn, torch

# Create a linear layer using pytorch's nn.Linear
lin = nn.Linear(in_features=3, out_features=2)

x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))


### 2.2 - A `Transformer` Layer

Most LLMs are a **stack of identical Transformer blocks**. Each block fuses two main components:

| Step | What it does | Where it lives in code |
|------|--------------|------------------------|
| **Multi-Head Self-Attention** | Every token looks at every other token and decides *what matters*. | `block.attn` |
| **Feed-Forward Network (MLP)** | Re-mixes information token-by-token. | `block.mlp` |

In the next section, you will load `GPT-2` and inspect its first Transformer block to see these components in a real model. You will locate its layers, print their shapes and parameters, and understand how a block processes a batch of token embeddings.

In [ ]:
import torch
from transformers import GPT2LMHeadModel

# Step 1: load the smallest GPT-2 model (124M parameters) using the Hugging Face transformers library.
# Refer to: https://huggingface.co/docs/transformers/en/model_doc/gpt2

# Step 1: Load the smallest GPT-2 model (124M parameters)
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Step 2: Inspect the first Transformer block
print(model.transformer.h[0])

# Print the full model architecture
print(model)

# Print key configuration details
print("\nModel Configuration:")
print(f"Number of layers: {model.config.n_layer}")
print(f"Hidden size: {model.config.n_embd}")
print(f"Number of attention heads: {model.config.n_head}")
print(f"Vocabulary size: {model.config.vocab_size}")

# Count total parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

Explanation of Code: GPT2LMHeadModel.from_pretrained("gpt2")  loads the 124M parameter GPT-2 model from Hugging Face.

 model.transformer.h  is a list of Transformer blocks.

 model.transformer.h[0]  prints the first block, which includes:
Multi-head attention

Feed-forward layers

Layer normalization

In this section, you will run a minimal forward pass through one GPT-2 block to understand how tokens are transformed inside the model.

In [ ]:
import torch
from transformers import GPT2LMHeadModel

# Step 0: Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Step 1: Create a small dummy input with a sequence of 8 random token IDs
batch_size = 1
seq_len = 8
dummy_ids = torch.randint(
    low=0,
    high=model.config.vocab_size,
    size=(batch_size, seq_len),
    device=device
)
print("Dummy token IDs:", dummy_ids)

# Step 2: Convert token IDs into embeddings (token + position)
# Ensure position ids are on the same device and of dtype long
position_ids = torch.arange(seq_len, device=device, dtype=torch.long).unsqueeze(0)  # shape: (1, seq_len)

with torch.no_grad():
    token_embeddings = model.transformer.wte(dummy_ids)       # (B, T, hidden_size)
    position_embeddings = model.transformer.wpe(position_ids) # (1, T, hidden_size)
    hidden_states = token_embeddings + position_embeddings     # (B, T, hidden_size)

# Build a causal attention mask of shape (B, 1, T, T)
# Upper-triangular with -inf above diagonal, zeros on/under diagonal
T = seq_len
causal_mask = torch.full((T, T), float("-inf"), device=device)
causal_mask = torch.triu(causal_mask, diagonal=1)  # keep -inf above the diagonal
causal_mask = causal_mask.unsqueeze(0).unsqueeze(0)  # (1, 1, T, T)
causal_mask = causal_mask.expand(batch_size, 1, T, T)  # (B, 1, T, T)

# Step 3: Pass the embeddings through a single Transformer block (the first block, h[0])
block = model.transformer.h[0]
block_out = block(
    hidden_states,            # (B, T, hidden_size)
    layer_past=None,
    attention_mask=causal_mask,  # (B, 1, T, T)
    use_cache=False,
    output_attentions=False
)[0]  # the first element is the new hidden_states

# Step 4: Inspect the result
print("Output shape after 1st block:", block_out.shape)  # (batch_size, seq_len, hidden_size)

In [ ]:
# Step 1: Create a small dummy input with a sequence of 8 random token IDs.
batch_size = 1
seq_len = 8
dummy_ids = torch.randint(low=0, high=model.config.vocab_size, size=(batch_size, seq_len))
print("Dummy token IDs:", dummy_ids)

# Step 2: Convert token IDs into embeddings
# GPT-2 uses two embedding layers:
#   - wte (word token embeddings)
#   - wpe (positional embeddings)
# Add them together to form the initial hidden representation of your input tokens.
# GPT-2 uses two embedding layers: wte (word embeddings) and wpe (positional embeddings)
token_embeddings = model.transformer.wte(dummy_ids)  # shape: (batch_size, seq_len, hidden_size)
position_ids = torch.arange(seq_len).unsqueeze(0)  # shape: (1, seq_len)
position_embeddings = model.transformer.wpe(position_ids)
hidden_states = token_embeddings + position_embeddings  # initial hidden representation

# Step 3: Pass the embeddings through a single Transformer block
# This simulates one layer of computation in GPT-2.
output = model.transformer.hhidden_states

# Step 4: Inspect the result
# The output shape should be (batch_size, sequence_length, hidden_size)
print("Output shape:", output.shape)  # should be (batch_size, seq_len, hidden_size)


In [ ]:
import torch
from transformers import GPT2LMHeadModel

# Step 0: Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

# Step 1: Create a small dummy input with a sequence of 8 random token IDs
batch_size = 1
seq_len = 8
dummy_ids = torch.randint(
    low=0,
    high=model.config.vocab_size,
    size=(batch_size, seq_len)
)
print("Dummy token IDs:", dummy_ids)

# Step 2 & 3: Run forward with hidden states
with torch.no_grad():
    outputs = model(
        input_ids=dummy_ids,
        output_hidden_states=True,
        return_dict=True
    )

# outputs.hidden_states is a tuple of length n_layer + 1:
# [0] = embeddings output (after wte + wpe)
# [1] = after block 0
# ...
# [n_layer] = after last block
first_block_output = outputs.hidden_states[1]

# Step 4: Inspect the result
print("Output shape after 1st block:", first_block_output.shape)  # (batch_size, seq_len, hidden_size)

✅ Explanation:

Step 1:  torch.randint  generates random token IDs in the GPT-2 vocabulary range.

Step 2:
 wte  converts token IDs to word embeddings.

 wpe  adds positional embeddings for sequence order.

Sum them to get the initial hidden representation.

Step 3: Pass through the first Transformer block ( h[0] ).

Step 4: Print the shape →  (1, 8, 768)  for GPT-2 small.

### 2.3 - Inside GPT-2

GPT-2 is essentially a stack of identical Transformer blocks arranged in sequence.
Each block contains attention, feed-forward, and normalization layers that process token representations step by step.

In this section, you will print the modules inside the GPT-2 Transformer to see how these components are organized.
This will help you understand how the model scales from a single block to a full network of many layers working together.

✅ Explanation:

 named_children()  returns an iterator over immediate child modules of  model.transformer  with their names.

 name  → the layer name (e.g.,  wte ,  wpe ,  drop ,  h ,  ln_f ).

 layer  → the actual PyTorch module.



If you want all sublayers including inside each Transformer block, you can use  named_modules() :


for name, layer in model.transformer.named_modules():
    print(name)


This will print a hierarchical list of every module inside GPT-2’s transformer stack.

In [ ]:
# Print the name of all layers inside gpt.transformer.
# You may find this helpful: https://docs.pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.named_children

for name, layer in model.transformer.named_children():
    print(name)

As you can see, the Transformer holds various modules, arranged from a list of blocks (`h`). The following table summarizes these modules:

| Step | What it does | Why it matters |
|------|--------------|----------------|
| **Token → Embedding** | Converts IDs to vectors | Gives the model a numeric “handle” on words |
| **Positional Encoding** | Adds “where am I?” info | Order matters in language |
| **Multi-Head Self-Attention** | Each token asks “which other tokens should I look at?” | Lets the model relate words across a sentence |
| **Feed-Forward Network** | Two stacked Linear layers with a non-linearity | Mixes information and adds depth |
| **LayerNorm & Residual** | Stabilize training and help gradients flow | Keeps very deep networks trainable |


Great question! Let’s break down each term clearly:

---

### ✅ **What is a Tensor?**
- A **tensor** is a generalization of vectors and matrices to higher dimensions.
- In PyTorch (and most deep learning frameworks), a tensor is:
  - A **multi-dimensional array** that can store numbers.
  - Similar to a NumPy array, but optimized for GPU acceleration and automatic differentiation.

Examples:
- Scalar → 0D tensor (e.g., `torch.tensor(5)`)
- Vector → 1D tensor (e.g., `[1, 2, 3]`)
- Matrix → 2D tensor (e.g., `[[1, 2], [3, 4]]`)
- Higher dimensions → 3D, 4D tensors (used for images, sequences, etc.)

---

### ✅ **What are Logits?**
- **Logits** are the raw, unnormalized outputs of a neural network **before applying softmax**.
- They represent scores for each class or token.
- For language models:
  - Each position in the sequence has a vector of logits (size = vocabulary size).
  - Higher logits → higher likelihood after applying softmax.

Example:
If the vocabulary size is 50,000, then for one token position, the model outputs **50,000 logits** (one per token).

---

### ✅ **What is a Tensor of Logits?**
- It’s a tensor that contains logits for **all tokens in the sequence**.
- Shape for GPT-2:
  ```
  (batch_size, sequence_length, vocab_size)
  ```
Example:
- `batch_size = 1`
- `sequence_length = 8`
- `vocab_size = 50,257`
Then logits tensor shape = `(1, 8, 50257)`.

This means:
- For each of the 8 positions in the input sequence, the model predicts a score for every token in the vocabulary.

---

✅ **Next step suggestion:** Do you want me to **show a quick visualization of how logits → probabilities → predicted token using softmax and top-k selection?**

### 2.4 LLM's output

When you pass a sequence of tokens through a language model, it produces a tensor of logits with shape
`(batch_size, seq_len, vocab_size)`.
Each position in the sequence receives a vector of scores representing how likely every possible token is to appear next. By applying a softmax function on the last dimension, these logits can be converted into probabilities that sum to 1.

In the next cell, you will feed an 8-token dummy sequence into GPT-2, print the shape of its logits, and display the five most likely next tokens predicted for the final position in the sequence.


In [ ]:
import torch, torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# Step 1: Load GPT-2 model and its tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
# Step 2: Tokenize input text
text = "Hello my name"

inputs = tokenizer(text, return_tensors="pt")

In [ ]:
# Step 3: Pass the input IDs to the model
outputs = model(**inputs)
logits = outputs.logits

In [ ]:
# Step 4: Predict the next token
# We take the logits from the final position, apply softmax to get probabilities,
# and then extract the top 5 most likely next tokens. You may find F.softmax and torch.topk helpful in your implementation.
# Step 4: Predict the next token
# Take logits from the last position
last_token_logits = logits[0, -1, :]  # shape: (vocab_size,)

# Apply softmax to get probabilities
probs = F.softmax(last_token_logits, dim=-1)

# Get top 5 most likely tokens
top_k = torch.topk(probs, k=5)
top_ids = top_k.indices
top_probs = top_k.values

# Decode the tokens
top_tokens = [tokenizer.decode([token_id]) for token_id in top_ids]

print("Top 5 predictions:")
for token, prob in zip(top_tokens, top_probs):
    print(f"{token} (prob: {prob:.4f})")

### 2.5 - Key Takeaway

A language model is not a black box or something mysterious.
It is a large composition of simple, understandable layers such as linear layers, attention, and normalization, trained together to predict the next token in a sequence.

By learning this next-token prediction task at scale, the model gradually develops an internal understanding of language structure, meaning, and context, which allows it to generate coherent and relevant text.

# 3 - Text Generation (Decoding)
Once a language model has been trained to predict token probabilities, we can use it to generate text.
This process is called text generation or decoding.

At each step, the model outputs a probability distribution over possible next tokens.
A decoding algorithm then selects one token based on that distribution, appends it to the sequence, and repeats the process to build text word by word. Different decoding strategies control how the model chooses the next token and how creative or deterministic the output will be. For example:
- **Greedy** decoding: always pick the token with the highest probability. Simple and consistent, but often repetitive.
- **Top-k** or **Nucleus** (top-p) sampling: randomly sample from the top few likely tokens to add variety.
- Beam search: explores multiple candidate continuations and keeps the best overall sequence.

Note: `Temperature` adjusts randomness in sampling. Higher values make outputs more diverse, while lower values make them more focused and deterministic.

### 3.1 - Greedy decoding
In this section, you will use GPT-2 and Hugging Face's built-in generate method to produce text using the greedy decoding strategy.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "gpt2"
device = "cuda" if torch.cuda.is_available() else "mps"

# Step 1. Load GPT-2 model and tokenizer.
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

# Step 2. Implement a text generation function using HuggingFace's generate method.
def generate(model, tokenizer, prompt, max_new_tokens=128):
    # Encode the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate text
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,        # enables sampling for creative text
        temperature=0.7,       # controls randomness
        top_k=50,              # limits sampling to top-k tokens
        top_p=0.95             # nucleus sampling
    )

    # Decode and return generated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n GPT-2 | Greedy")
    print(generate(model, tokenizer, prompt, 80))

Naively selecting the single most probable token at each step (known as greedy decoding) often leads to poor results in practice:
- Repetition loops: phrases like “The cat is is is…”
- Short-sighted choices: the most likely token right now might lead to incoherent text later

These issues are why more advanced decoding methods such as top-k and nucleus sampling are commonly used to make model outputs more diverse and natural.

### 3.2 - Top-k and top-p sampling
The generate function you implemented earlier can easily be extended to use different decoding strategies.

In this section, you will reimplement the same function but adapt it to support Top-k and Top-p (nucleus) sampling. These methods introduce controlled randomness, allowing the model to explore multiple plausible continuations instead of always choosing the single most likely next token.

In [ ]:
# Implement `generate` to support 3 strategies: greedy, top_k, and top_o
# You may find this link helpful: https://huggingface.co/docs/transformers/en/main_classes/text_generation

def generate(model, tokenizer, prompt, strategy="greedy", max_new_tokens=128):
    # Encode the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    strategy = strategy.strip().lower()
    print("strategy :", strategy)
    # Set generation parameters based on strategy
    if strategy == "greedy":
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False  # Greedy = no sampling
        )
    elif strategy == "top_k":
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,   # Enable sampling
            top_k=50,         # Sample from top 50 tokens
            temperature=0.7    # Add some randomness
        )
    elif strategy == "top_p":
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,   # Enable sampling
            top_p=0.9,        # Nucleus sampling (90% cumulative probability)
            temperature=0.7
        )
    else:
        raise ValueError("Invalid strategy. Choose from: 'greedy', 'top_k', 'top_p'.")

    # Decode and return generated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

### 3.3 - Try It Yourself

Now it’s time to experiment with text generation. Replace the sample prompts with your own prompts or adjust the decoding strategy.
You can experiment with:
- strategy: "greedy", "beam", "top_k", "top_p"
- temperature: values between 0.2 and 2.0
- k or p: thresholds that control sampling diversity

Try generating the same prompt with `greedy` and `top_p` (for example, 0.9). Notice how even small temperature changes can make the output more focused or more free-form.




In [ ]:

tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n GPT-2 | Top-p")
    print(generate(model, tokenizer, prompt, "top_k", 40))

# 4 - Completion vs. Instruction-tuned LLMs

So far, we have used `GPT-2` to generate text from a given input prompt. However, `GPT-2` is just a completion model. It simply continues the provided text without understanding it as a task or question. It is not designed to engage in dialogue or follow instructions.

In contrast, instruction-tuned LLMs (such as `Qwen-Chat`) undergo an additional post-training stage after base pre-training. This process fine-tunes the model to behave helpfully and safely when interacting with users. Because of this extra stage, instruction-tuned models can:

- Interpret prompts as requests rather than just text to continue
- Stay in conversation mode, answering questions and following steps
- Handle refusals and safety boundaries appropriately
- Maintain a consistent helpful persona, rather than drifting into storytelling

### 4.1 - `Qwen/Qwen3-0.6B` vs. `GPT2`

In the next cell, you will feed the same prompt to two different models:

- GPT-2 (completion-only): continues the text in the same writing style
- Qwen/Qwen3-0.6B (instruction-tuned): interprets the input as an instruction and responds helpfully

Comparing the two outputs will make the difference between completion and instruction-tuned behavior clear.



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer and model
tok = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)
model.eval()

# Prepare input
inputs = tok("The secret to baking a good cake is ", return_tensors="pt").to(device)

# Set up streaming (prints tokens as they form words)
streamer = TextStreamer(tok)

# Generate text
_ = model.generate(**inputs, max_new_tokens=50, streamer=streamer)

We have now downloaded two small checkpoints: GPT-2 (124M parameters) and Qwen3-0.6B (600M parameters). If the previous cell took some time to run, that was mainly due to model download speed. The models will be cached locally, so future runs will be faster.

Next, we will generate text using our generate function with both models and the same prompt to directly compare how a completion-only model (GPT-2) behaves differently from an instruction-tuned model (Qwen).

In [ ]:

tests=[("Once upon a time", "greedy"),("What is 2+2?", "top_k"),("Suggest a party theme.", "top_p")]

for prompt, strategy in tests:
    # Tokenize input
    inputs = gpt2_tokenizer(prompt, return_tensors="pt").to(device)

    # Generate based on strategy
    if strategy == "greedy":
        outputs = gpt2_model.generate(**inputs, max_new_tokens=30)
    elif strategy == "top_k":
        outputs = gpt2_model.generate(
            **inputs,
            max_new_tokens=30,
            do_sample=True,
            top_k=50
        )
    elif strategy == "top_p":
        outputs = gpt2_model.generate(
            **inputs,
            max_new_tokens=30,
            do_sample=True,
            top_p=0.9
        )

    # Print results
    print(f"\nPrompt: {prompt}")
    print(f"Strategy: {strategy}")
    print(f"Generated: {gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)}")

# 5. (Optional) A Small Interactive LLM Playground
This section is optional. You do not need to implement it to complete the project. It is meant purely for exploration and will not significantly affect your core AI engineering skills.

If you are curious, you can build a simple interactive playground to experiment with text generation. You can:
- Create input widgets for the prompt, model selection, decoding strategy, and temperature
- Use Hugging Face's generate method to produce text based on the selected settings
- Display the model's response directly in the notebook output

You may find following links helpful:
- https://ipywidgets.readthedocs.io/en/latest/
- https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Steps to implement:
# 1. Load models and tokenizers (GPT-2 and Qwen).
# 2. Define a helper function to generate text with different decoding strategies.
# 3. Create interactive UI elements (prompt box, model selector, strategy selector, temperature slider).
# 4. Add a button to trigger text generation.
# 5. Define the button’s behavior.
# 6. Display the full UI for the playground.

# 1. UI elements
prompt_box = widgets.Textarea(
    value="Once upon a time",
    placeholder="Enter your prompt here...",
    description="Prompt:",
    layout=widgets.Layout(width="80%", height="80px")
)

model_selector = widgets.Dropdown(
    options=["GPT-2", "Qwen"],
    value="GPT-2",
    description="Model:"
)

strategy_selector = widgets.Dropdown(
    options=["greedy", "top_k", "top_p"],
    value="greedy",
    description="Strategy:"
)

temperature_slider = widgets.FloatSlider(
    value=0.7, min=0.1, max=1.5, step=0.1,
    description="Temperature:"
)

generate_button = widgets.Button(description="Generate Text", button_style="success")

# 2. Display UI
display(Markdown("### 📝 Text Generation Playground"))
display(prompt_box, model_selector, strategy_selector, temperature_slider, generate_button)

# ---------------------------------------------
# (Optional) Step 2 helper: uncomment if you want a reusable function
# def generate_text(model, tokenizer, prompt, strategy="greedy", temperature=0.7,
#                   max_new_tokens=128, top_k=50, top_p=0.9):
#     import torch
#     inputs = tokenizer(prompt, return_tensors="pt")
#     gen_args = {"max_new_tokens": max_new_tokens}
#     if strategy == "greedy":
#         gen_args.update({"do_sample": False})
#     elif strategy == "top_k":
#         gen_args.update({"do_sample": True, "top_k": top_k, "temperature": temperature})
#     elif strategy == "top_p":
#         gen_args.update({"do_sample": True, "top_p": top_p, "temperature": temperature})
#     else:
#         raise ValueError(f"Unknown strategy: {strategy}")
#     with torch.no_grad():
#         output_ids = model.generate(**inputs, **gen_args)
#     text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
#     return text
# ---------------------------------------------

# === Step 3. Add an output display area ===
output_area = widgets.Output(layout=widgets.Layout(border="1px solid #ddd", padding="10px", width="80%"))
status_label = widgets.HTML("")

# === Step 4. Button to trigger text generation (already created as generate_button) ===
# We’ll just wire it up here.

# === Step 5. Define the button’s behavior ===
def on_generate_clicked(_):
    output_area.clear_output()
    status_label.value = "⏳ Generating…"

    # Pull current UI selections
    prompt = prompt_box.value.strip()
    model_name = model_selector.value
    strategy = strategy_selector.value
    temperature = float(temperature_slider.value)

    if not prompt:
        status_label.value = "⚠️ Please enter a prompt."
        return

     # Build decoding args based on strategy
    gen_args = {"max_new_tokens": 128}
    if strategy == "greedy":
        gen_args.update({"do_sample": False})
    elif strategy == "top_k":
        gen_args.update({"do_sample": True, "top_k": 50, "temperature": temperature})
    elif strategy == "top_p":
        gen_args.update({"do_sample": True, "top_p": 0.9, "temperature": temperature})
    else:
        status_label.value = f"⚠️ Unknown strategy: {strategy}"
        return

    try:
        import torch
        inputs = tokenizer(prompt, return_tensors="pt")
        with torch.no_grad():
            output_ids = model.generate(**inputs, **gen_args)
        full_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # Show only the continuation (after the prompt) if possible
        if full_text.startswith(prompt):
            continuation = full_text[len(prompt):].lstrip("\n")
        else:
            continuation = full_text

        with output_area:
            display(Markdown(f"Prompt\n\n> {prompt}"))
            display(Markdown("---"))
            display(Markdown("Generated Text"))
            display(Markdown(f"{continuation if continuation else full_text}"))

        status_label.value = "✅ Done."
    except Exception as e:
        status_label.value = "❌ Error while generating."
        with output_area:
            print(repr(e))

# Wire up the button
generate_button.on_click(on_generate_clicked)


## 🎉 Congratulations!

You've just learned, explored, and inspected a real **LLM**. In one project you:
* Learned how **tokenization** works in practice
* Used `tiktoken` library to load and experiment with most advanced tokenizers.
* Explored LLM architecture and inspected GPT2 blocks and layers
* Learned decoding strategies and used `top-p` to generate text from GPT2
* Loaded a powerful chat model, `Qwen3-0.6B` and generated text
* Built an LLM playground


👏 **Great job!** Take a moment to celebrate. You now have a working mental model of how LLMs work. The skills you used here power most LLMs you see everywhere.
